# Initial data

In [58]:
import requests, json, datetime, pandas as pd

# Данные тестовой среды для подключения
token_url = "http://mosrepora02.evropolis.ru:8080/ords/rest_api/oauth/token"
test_api_url = "http://mosrepora02.evropolis.ru:8080/ords/rest_api/api"

#Список данных для Oauth2
users = {
    'che': {
            'id': 'xDZqGTn_qUyGJzuwZnmOzQ..',
            'secret': 'Hyt09zV2nql4hG23-1tOyw..'
        },
    'banki': {
            'id': '_FBN9r-0SXzhlWJ7EBNEvQ..',
            'secret': 'tTuR05XZfcpBBo_tVXWy0A..'
        }    
    }

#users['che']['id']

# Данные пользователя для создания
def NewSubject():
    Subject = {
                "lastname": "Тестовый", # фамилию не меняем, так как она используется в тесте на дубли е-ё-симв,
                "firstname": "Тккевтеуррр",
                "parentname": "Тестович",
                "sex": "М",
                "birthday": "1983-05-24T00:00:00",
                "latinname": "Gridnev Roman",
                "drivingdatebeg": "2004-01-01T00:00:00",
                "citizenship": "RUS",
                "address": [
                    {
                        "code": 2247,
                        "code_desc": "Регистрация",
                        "text": "Московская обл, Мытищи г, Воровского ул, д. 5 кв 21",
                        "street_kladr_id": "50000044000005900"
                    }
                ],
                "contact": [
                    {
                        "code": 2243,
                        "code_desc": "E-mail личный",
                        "text": "test@ya.ru"
                    },
                                    {
                        "code": 2240,
                        "text": "+7(926) 449-48-60"
                     },

                ],
                "document": [

                    {
                        "code": 1165,
                        "code_desc": "Паспорт гражданина России",
                         "series": "1111",
                         "number": "111111",
                        "issue_date": "2003-08-12T00:00:00",
                        "issue_by": "ОВД ОВД ОВД"
                    },
                    {
                    "code": 1145,
                    "code_desc": "Водительское удостоверение",
                    "series": "50ОТ",
                    "number": "432259",
                    "issue_date": "2018-01-25T00:00:00",
                    "issue_by": "ГИБДД" 
                    }
               ]
            }
    return Subject

# Main functions and classes

In [32]:
# Функция получения токена Oauth 2
def GetToken(user):
    data = {'grant_type': 'client_credentials'}
    client_id = users[user]['id']
    client_secret = users[user]['secret']
    access_token_response = requests.post(token_url, data=data, verify=False, allow_redirects=False, auth=(client_id, client_secret))
    tokens = json.loads(access_token_response.text)
    return tokens['access_token']

class apirequest():
    """docstring"""
    def __init__(self, user):
        self.token = GetToken(user)
        self.api_call_headers = {'Authorization': 'Bearer ' + self.token}
        
    def getsubject(self, subjisn):
        resp = []
        timebeg = datetime.datetime.now()
        api_call_response = requests.get(test_api_url + '/subject/' + str(subjisn), headers=self.api_call_headers, verify=False)
        subject = json.loads(api_call_response.text)
        timeend = datetime.datetime.now()
        
        resp.append(api_call_response.status_code)
        resp.append((timeend - timebeg).microseconds/1000)
        resp.append(json.loads(api_call_response.text))
        return resp
    
    def newsubject(self, subject):
        resp = []
        timebeg = datetime.datetime.now()
        api_call_response = requests.post(test_api_url + '/subject', json = subject, headers=self.api_call_headers, verify=False)
        subject = json.loads(api_call_response.text)
        timeend = datetime.datetime.now()
        
        resp.append(api_call_response.status_code)
        resp.append((timeend - timebeg).microseconds/1000)
        resp.append(json.loads(api_call_response.text))
        return resp

In [61]:
GetToken('banki')

'r5oeW3mmilKbz0TtC4_K8g'

In [63]:
%%time
api = apirequest('banki')
Subject = NewSubject()
api.newsubject(Subject)
#api.getsubject(446014414)
#NewSubject

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

# Getsubject load test 

In [48]:
test_users = pd.read_csv('test_users.csv')

In [130]:
results = pd.DataFrame(columns = ['user', 
                                  'resp_time_ms',
                                  'resp_status_code',
                                  'resp_status_name',
                                  'resp_status_error',
                                  'subject_isn'])
results

,user,resp_time_ms,resp_status_code,resp_status_name,resp_status_error,subject_isn


In [133]:
n=0
timebeg = datetime.datetime.now()
for user in users.keys():
    api = apirequest(user)
    for isn in test_users:
        resp = api.getsubject(isn)
        results.loc[n] = {'user': user, 
                          'resp_time_ms': resp[1], 
                          'resp_status_code': resp[0], 
                          'resp_status_name': resp[2]['status']['code'], 
                          'resp_status_error': resp[2]['status']['message'], 
                          'subject_isn': isn}
        n += 1
timeend = datetime.datetime.now()
print('Время, затраченное на тест:', timeend-timebeg)       
    

время, затраченное на тест: 0:00:07.736359


In [132]:
results

,user,resp_time_ms,resp_status_code,resp_status_name,resp_status_error,subject_isn
0,che,660.227,200,success,None,445903407
1,che,442.125,200,success,None,445903412
2,che,358.076,200,success,None,445903414
3,banki,417.876,200,success,None,445903407
4,banki,411.895,200,success,None,445903412
5,banki,512.661,200,success,None,445903414


In [127]:
results.agg({'resp_time_ms':['count', 'min', 'max', 'median', 'mean']})

,resp_time_ms
count,6.000000
min,85.049000
max,933.422000
median,117.948500
mean,352.150833


# Newsubject functionality test

In [85]:
def newsubjecttest(user, details=False):
    api = apirequest(user)
    timebeg = datetime.datetime.now()
    row = 1

    result = pd.DataFrame(columns = ['case', 
                                      'timeelapsed',
                                     'resp_html_code',
                                      'resp_status_code',
                                      'resp_status_error',
                                      'created_subjisn',
                                      'test_result'
                                    ])

    def writeresult(row, resp, created_subjisn, test_result):
        result.loc[row] = {'case': case, 
                            'timeelapsed': resp[1], 
                            'resp_html_code': resp[0], 
                            'resp_status_code': resp[2]['status']['code'], 
                            'resp_status_error': resp[2]['status']['message'],
                            'created_subjisn': created_subjisn,
                            'test_result': test_result
                           }




    case = 'Минимальный набор данных'
    if details == True:
        print (case)
    testsubject = NewSubject()
    del testsubject['parentname']
    del testsubject['sex']
    del testsubject['latinname']
    del testsubject['drivingdatebeg']
    del testsubject['citizenship']
    del testsubject['address']
    del testsubject['contact']
    del testsubject['document']
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None:
        created_subjisn = None
        test_result = 'Fail'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Ok'

    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Два паспорта'
    if details == True:
        print (case)
    testsubject = NewSubject()
    testsubject['document'] = testsubject['document']*2
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'

    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Два адреса'
    if details == True:    
        print (case)
    testsubject = NewSubject()
    testsubject['address'] = testsubject['address']*2
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'

    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Документы из прошлого'
    if details == True:
        print (case)
    testsubject = NewSubject()
    testsubject['document'][0]['issue_date'] = "1883-05-24"
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Документы из будущего'
    if details == True:
        print (case)
    testsubject = NewSubject()
    testsubject['document'][0]['issue_date'] = "2883-05-24"
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'    
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Паспорт выдан до 14 лет'
    if details == True:
        print (case)
    testsubject = NewSubject()
    testsubject['document'][0]['issue_date'] = "1984-05-24"
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'    
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Телефон не нужного формата'
    if details == True:
        print (case)
    testsubject = NewSubject()
    testsubject['contact'][1]['text'] = "1984-05-24"
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'    
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Паспорт без серии и номера'
    if details == True:
        print (case)
    testsubject = NewSubject()
    del testsubject['document'][0]['series']
    del testsubject['document'][0]['number']
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Ok'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Fail'    
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Паспорт только серия и номер'
    if details == True:
        print (case)
    testsubject = NewSubject()
    del testsubject['document'][0]['code_desc']
    del testsubject['document'][0]['issue_date']
    del testsubject['document'][0]['issue_by']
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Fail'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        test_result = 'Ok'    
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Попытка создания дубля'
    if details == True:
        print (case)
        print ('Исходный контрагент')
    testsubject = NewSubject()
    if details == True:
        print (testsubject)
    resp = api.newsubject(testsubject)
    # обрабатываем случай, когда результат не вернулся
    if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
        created_subjisn = None
        test_result = 'Fail'
    else:
        created_subjisn = resp[2]['result']['data']['id']
        if details == True:
            print(testsubject)
            print('Новый контрагент: ', created_subjisn)
        testsubject = NewSubject()
        if details == True:
            print('----------')
            print ('Проверяем кейс с дублем по номеру телефона')
        del testsubject['sex']
        del testsubject['latinname']
        del testsubject['drivingdatebeg']
        del testsubject['citizenship']
        del testsubject['address']
        del testsubject['document']
        resp2 = api.newsubject(testsubject)
        created_subjisn2 = resp2[2]['result']['data']['id']
        if details == True:
            print(testsubject)
            print('Новый контрагент: ', created_subjisn2)
        testsubject = NewSubject()
        if details == True:
            print('----------')
            print ('Проверяем кейс с дублем по номеру паспорта и ВУ')
        del testsubject['sex']
        del testsubject['latinname']
        del testsubject['drivingdatebeg']
        del testsubject['citizenship']
        del testsubject['address']
        del testsubject['contact']
        resp3 = api.newsubject(testsubject)
        created_subjisn3 = resp3[2]['result']['data']['id']
        if details == True:
            print(testsubject)
            print('Новый контрагент: ', created_subjisn3)
        testsubject = NewSubject()
        if details == True:
            print('----------')
        testsubject = NewSubject()
        if details == True:
            print('----------')
            print ('Проверяем кейс с символами и е-ё в фамилии')
        testsubject["lastname"] = "Тёстовыи "
        resp5 = api.newsubject(testsubject)
        created_subjisn5 = resp5[2]['result']['data']['id']
        if details == True:
            print(testsubject)
            print('Новый контрагент: ', created_subjisn2)
            print ('Проверяем кейс с дублем по ФИО и дате рождения')
        del testsubject['sex']
        del testsubject['latinname']
        del testsubject['drivingdatebeg']
        del testsubject['citizenship']
        del testsubject['address']
        del testsubject['contact']
        del testsubject['document']
        resp4 = api.newsubject(testsubject)
        resp6 = api.newsubject(testsubject)
        created_subjisn4 = resp4[2]['result']['data']['id']
        created_subjisn6 = resp6[2]['result']['data']['id']
        if details == True:
            print(testsubject)
            print('Новый контрагент: ', created_subjisn4)
            print('Новый контрагент: ', created_subjisn6)
        if created_subjisn4 == created_subjisn6:
            if created_subjisn == created_subjisn2 == created_subjisn3 == created_subjisn5:
                test_result = 'Ok'
        else:
            test_result = 'Fail'
    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    case = 'Многократная отправка данных'
    if details == True:
        print (case)
    testsubject = NewSubject()
    if details == True:
        print (testsubject)
    i = 0
    isn = set()
    while i <= 10:
        resp = api.newsubject(testsubject)
        # обрабатываем случай, когда результат не вернулся
        if resp[2].get('result') == None or resp[2].get('result') == {'data': None}:
            created_subjisn = resp[2]['result']['data']['id']
            test_result = 'Fail'
            created_subjisn = None

        else:
            created_subjisn = resp[2]['result']['data']['id']
            isn.add(created_subjisn)
        i += 1
    if len(isn) == 1:
        test_result = 'Ok'
    else:
        test_result = 'Fail'
    if details == True:
        print (str(isn))

    writeresult(row, resp, created_subjisn, test_result)
    row += 1

    timeend = datetime.datetime.now()
    if details == True:
        print('----------')
        print('----------')
        print('Время, затраченное на тест:', timeend-timebeg)
        return result
    else:
        if 'Fail' in str(result['test_result']):
            return 'Fail'
        else:
            return 'Ok'

In [68]:
result

,case,timeelapsed,resp_html_code,resp_status_code,resp_status_error,created_subjisn,test_result
1,Минимальный набор данных,229.710,200,success,None,446074420,Ok
2,Два паспорта,682.177,200,error,"[{'code': 20285, 'type': 'error', 'text': 'Для...",None,Ok
3,Два адреса,388.958,200,error,"[{'code': 20206, 'type': 'error', 'text': 'Для...",None,Ok
4,Документы из прошлого,970.405,200,error,"[{'code': 20250, 'type': 'error', 'text': 'Для...",None,Ok
5,Документы из будущего,415.891,200,error,"[{'code': 20255, 'type': 'error', 'text': 'Для...",None,Ok
6,Паспорт выдан до 14 лет,649.265,200,error,"[{'code': 20250, 'type': 'error', 'text': 'Для...",None,Ok
7,Телефон не нужного формата,474.728,200,error,"[{'code': 20215, 'type': 'error', 'text': 'Для...",None,Ok
8,Паспорт без серии и номера,543.550,200,error,"[{'code': 20262, 'type': 'error', 'text': 'Для...",None,Ok
9,Паспорт только серия и номер,422.511,200,success,None,446074421,Ok
10,Попытка создания дубля,569.441,200,success,None,446074421,Ok


In [90]:
newsubjecttest('banki', details=False)

'Ok'